#### Imports

In [2]:
!ls

MURA-v1.1  sample_data


In [3]:
# -*- coding: utf-8 -*-
#!wget -c https://cs.stanford.edu/group/mlgroup/MURA-v1.1.zip
#!unzip MURA-v1.1.zip
#!rm MURA-v1.1.zip
!ls

MURA-v1.1  sample_data


In [4]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
from tqdm import tqdm
import keras
pd.options.display.max_colwidth = 100
from keras.applications.mobilenetv2 import MobileNetV2
from keras.applications.densenet import DenseNet169
from keras.applications.inception_v3 import InceptionV3
from keras.applications.resnet50 import ResNet50
from keras.applications.nasnet import NASNetLarge
from keras.preprocessing import image
from keras.applications.mobilenetv2 import preprocess_input
from keras.applications import MobileNet
from keras.callbacks import (EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard)
from keras.layers import Dense, GlobalAveragePooling2D, Input
from keras.metrics import binary_accuracy, binary_crossentropy
from keras.models import Model
from keras.optimizers import SGD, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenetv2 import MobileNetV2
from keras.preprocessing import image as k_im_prep
from keras.models import Model
from keras import backend as K
from keras.models import Sequential

Using TensorFlow backend.


**Data Reading**

In [0]:
def paths_n_labels(csv,str_limp):
    #make dataframe
    studies=pd.read_csv(csv, sep=',',header=None)
    #separate study paths and labels of given limp from those of other limps
    limp_studies=studies[studies[0].str.contains(str_limp)==True]
    #make it a numpy
    limp_studies=np.array(limp_studies)
    #limp study folder paths
    limp_paths=[]
    #labels of given limp
    limp_labels=[]
    for i in tqdm( range(limp_studies.shape[0]) ):
        study_path=limp_studies[i][0]
        study_label=limp_studies[i][1]
        study_files = [f for f in listdir(study_path) if isfile(join(study_path, f))]
        for image in study_files:
            limp_paths.append(study_path + image)
            limp_labels.append(study_label)

    limp_paths=np.array(limp_paths)
    limp_labels=np.array(limp_labels)

    return limp_paths,limp_labels

In [0]:
#general function with options for wrist data case, only set wrist_train=True in the case of wrist training data only
#for all other limps and for validation data even that of wrist just pass the paths
#targ_size is image resizing with default(224,224)
#preprocess flag is for using keras preprocessing for images or just resizing
def read_images(paths ,targ_size= (224, 224), wrist_train=False, preprocess=False):
    images=[]
    #load any limp images
    if(not wrist_train):
        for path in tqdm(paths):
            img=k_im_prep.load_img(path, target_size=targ_size )
            if(preprocess):
                img = k_im_prep.img_to_array(img)
                img = np.expand_dims(img, axis=0)
                img = preprocess_input(img)
                images.append(np.array(img)[0])
            else:
              images.append(np.array(img))
    #special case for wrist train corrupted data       
    else:
        #did this because it gave an error at sample  5307 or near it if took all
        sample_e=5307
        sample_s2=5339
        images=[]
        for path in tqdm(paths[:sample_e]):
            img=k_im_prep.load_img(path, target_size=targ_size)
            if(preprocess):
                img = k_im_prep.img_to_array(img)
                img = np.expand_dims(img, axis=0)
                img = preprocess_input(img)
                images.append(np.array(img)[0])
            else:
              images.append(np.array(img))

        #new start
        for path in tqdm(paths[sample_s2:]):
            img=k_im_prep.load_img(path, target_size=targ_size)
            if(preprocess):
                img = k_im_prep.img_to_array(img)
                img = np.expand_dims(img, axis=0)
                img = preprocess_input(img)
                images.append(np.array(img)[0])
            else:
              images.append(np.array(img))

    #making it a numpy array instead of python list
    return (np.array(images))
  

def wrist_labels(labels):
  sample_e=5307
  sample_s2=5339
  return np.hstack( [ labels[:sample_e], labels[sample_s2:] ])


In [0]:
train_studies='MURA-v1.1/train_labeled_studies.csv'
valid_studies='MURA-v1.1/valid_labeled_studies.csv'

In [0]:
# #data bias : train
# print("0 normal, 1 abnormal")
# unique, counts = np.unique(train_labels, return_counts=True)
# print(dict(zip(unique, counts)))

In [0]:
# print("0 normal, 1 abnormal")
# unique, counts = np.unique(valid_labels, return_counts=True)
# print(dict(zip(unique, counts)))

### Model

In [0]:
 def images_n_labels(limp,preprocess=True):
  
  print("\nreading studies of "+ limp + "\n")
  print(train_studies)
  train_paths,train_labels=paths_n_labels(train_studies,limp)
  valid_paths,valid_labels=paths_n_labels(valid_studies,limp)
  
  print(train_labels.shape)
  print(valid_labels.shape)
  print("reading "+ limp + " training images")
  if (limp == "WRIST"):
    train_labels=wrist_labels(train_labels)
    train_imgs= read_images(train_paths,preprocess=preprocess, wrist_train=True)
    
  else:
    train_imgs= read_images(train_paths,preprocess=preprocess, wrist_train=False)
  print(train_imgs.shape)  
  print("reading "+ limp + " validation images")
  valid_imgs= read_images(valid_paths,preprocess=True)
  print(valid_imgs.shape)
  
  return train_imgs, train_labels, valid_imgs, valid_labels

In [0]:
def make_FT_model(base=1, imagenet=True, freeze_all=True, add_denses=True):
  
  #weights of pretrained model
  if (imagenet==True):
    w='imagenet'
  else:
    w=None
  
  #default because refrenced before assignment error, just scroll down
  base_model = MobileNetV2(input_shape= (224, 224, 3),weights=w, include_top=False)
  
  #initializing pretrained model
  if (base==0):
    base_model = MobileNetV2(input_shape= (224, 224, 3),weights=w, include_top=False)
  elif (base == 1):
    base_model = DenseNet169(input_shape= (224, 224, 3),weights=w, include_top=False)
  elif (base == 2):
    base_model = InceptionV3(input_shape= (224, 224, 3),weights=w, include_top=False)
  elif (base == 3):
    base_model = ResNet50(input_shape= (224, 224, 3),weights=w, include_top=False)   
  elif (base == 4):
    base_model = NASNetLarge(input_shape= (224, 224, 3),weights=w, include_top=False)
    
 
  if (freeze_all):
    #freeze layers of densenet
    for layer in base_model.layers:
      layer.trainable= False 
  
  # add a global spatial average pooling layer
  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  
  if(add_denses):
    # let's add a fully-connected layer
    #x = Dense(1024, activation='relu')(x)
    x = Dense(512, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    #x = Dense(32, activation='relu')(x)
    # and a logistic layer -- let's say we have 200 classes
    predictions = Dense(1, activation='sigmoid')(x)
    # this is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)
    
  else:
    # just feature extractor
    model = Model(inputs=base_model.input, output=x)
  
  
  return model

**The Generator**

TODO #K : the runtime died issue when trying to fit the generated data first before using it in fit_generator

In [0]:
datagen = ImageDataGenerator(  rescale=1./255,
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=20,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False,
    zoom_range=0.1,
    channel_shift_range=0.,
    fill_mode='nearest')

### Generalizing over all limps

In [0]:
#pass to model list of limps because if wanted to train on less
#function outputs a dictionary or dataframe has train and val accuracies for each limp using a chosen model
limps=["SHOULDER", "WRIST","FINGER", "ELBOW", "HUMERUS","HAND", "FOREARM"]


def evaluate_limps(models=[1], imagenet=True, freeze_all=False,v=1 , limps=["WRIST"], preprocess_ip=True, augment=False):
  accuracies={}
  for model in models:
    print("\n\n used base model: \n\n"+bases[model])
    for limp in limps:
      #print("reading "+ limp + " images\n")
      train_imgs, train_labels, valid_imgs, valid_labels= images_n_labels(limp, preprocess=preprocess_ip)
      print("making model")
      model=make_FT_model(base= model, imagenet=False, freeze_all=False, add_denses=True)
      print("compiling")
      model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
      ##############################################################

      if(augment): #dataaugmentation model fitting
        print("Augmenting Input data")
        # # compute quantities required for featurewise normalization
        # # (std, mean, and principal components if ZCA whitening is applied)

        #       datagen.fit(train_imgs)
        model.fit_generator(datagen.flow(train_imgs, train_labels, batch_size=16),
                      steps_per_epoch=len(train_imgs) / 16, epochs=5,use_multiprocessing=False,workers=6,validation_data=None)
        print("training augmentation calculations for "+ limp)

        loss_tr, accuracy_tr =model.evaluate_generator(datagen.flow(train_imgs,train_labels), use_multiprocessing=True,steps=len(datagen) / 16)

        print("calculating validation augmentation loss for "+ limp)

        loss_val, accuracy_val = model.evaluate_generator(datagen.flow(train_imgs,valid_labels), use_multiprocessing=True,steps=len(datagen) / 16)

      ###############################################################
      else:
        print("fitting")
        model.fit(train_imgs, train_labels, epochs=5, validation_data=(valid_imgs, valid_labels), shuffle=True, verbose=v )
        print("training loss calculations for "+ limp)
        
        loss_tr, accuracy_tr =model.evaluate(x=train_imgs, y=train_labels, batch_size=128, verbose=v)
        
        print("calculating validation loss for "+ limp)
        
        loss_val, accuracy_val =model.evaluate(x=valid_imgs, y=valid_labels, batch_size=128, verbose=v)
        
      accuracies.update( {limp : [accuracy_tr, accuracy_val]} )
      print("\n \n "+ limp)
      print(accuracies)
      pd.DataFrame(accuracies).head(2)
  return accuracies

  
  

In [0]:
#@title Attributes of Experiment { run: "auto", vertical-output: true, display-mode: "both" }
imagenet = True #@param {type:"boolean"}
#parameter setting for experiment
#choose here which limps to use when running the function
limps=["FINGER", "ELBOW", "HUMERUS","HAND", "FOREARM", "SHOULDER", "WRIST"]
# which Transfer Learning base models to use
base=[0]
#this is just to know index of each model when choosing base above and to print it when running the function
bases=["MobileNetV2", "DenseNet169", "InceptionV3", "ResNet50","NASNetLarge"]
# whether to use imagenet weights of not
imagenet=True
# whether to freeze ALL layers of base model or not
freeze_all=True
# how much information to display about epochs and progress , 0= none , 1 is line per epoch
verbose=1
# whether to preprocess input 
preprocess_ip=True
#whether to augment data
augmentation=False

In [0]:
R=evaluate_limps(models=base,imagenet=imagenet, freeze_all=freeze_all, v=verbose, limps=limps, preprocess_ip=preprocess_ip, augment=augmentation)


  2%|▏         | 46/1935 [00:00<00:04, 459.81it/s]



 used base model: 

MobileNetV2

reading studies of FINGER

MURA-v1.1/train_labeled_studies.csv


  0%|          | 9/5106 [00:00<00:56, 89.43it/s]

(5106,)
(461,)
reading FINGER training images


  4%|▎         | 17/461 [00:00<00:02, 165.29it/s]

...

  1%|          | 21/2821 [00:00<00:13, 206.94it/s]


 
 FOREARM
{'FINGER': [0.6549157851404458, 0.6811279839393632], 'ELBOW': [0.41188399917936197, 0.5161290323381783], 'HUMERUS': [0.47091194968553457, 0.4861111111111111], 'HAND': [0.7322749413674905, 0.5891304347826087], 'FOREARM': [0.36219178082191783, 0.5016611295681063]}

reading studies of SHOULDER

MURA-v1.1/train_labeled_studies.csv


  0%|          | 16/8379 [00:00<00:53, 156.75it/s]

(8379,)
(563,)
reading SHOULDER training images


100%|██████████| 8379/8379 [02:24<00:00, 57.94it/s]


In [0]:
pd.DataFrame(R).head(2)

In [0]:
#may need to make a function to evaluate all limps over one model together as one dataset

** Current point: generalizing functions and code cleaning+ seeing early stopping callback**

All unsolved problems:

* which layers to freeze and which to train + should I train TL before freezing it ?
* data augmentation to generate more data (solved by khaled,still needs little verification)
* recording variation in accuracy after every change to get intuition
* what does outputted loss represent ? how to read the number ?
* normalization step and its effect on accuracy
* should I use 1 or two neurons at output layer ?
* binary crossentropy weights
* justifying parameter use and discovering useful params
* try training with model unfrozen with imagenet and without it
* make a function to record and tabulate outputs
* could we add precision or recall metric ? change accuracy?
* why doesn't it work if removed GlobalAveragePooling line?
* see if want to freeze less layers
* generalize file reading functions
* use better batch size ? increase epochs ?
* training function with preprocessing flag and multiple model comparisons
* grid search like function to tune models and hyperparameters
* early stopping callback and save model to drive for continuing training
* feature concatenation
* ensemble model
*maybe we are reading images all wrong and we should read them to grayscale then on preprocessing return them to 3 channels

**results on wrist data: no preprocessing but resize**


1- Densenet, with imagenet and froze all at rms prop

59% train 55% val

2- Densenet with imagenet and didnt freeze (trained over them)

65% train and 65% validation

3-InceptionV3, with imagenet and froze all at rms prop

59% train 55% val

4-InceptionV3 with imagenet and didnt freeze (trained over them)

82% train and 78% validation



**results on wrist data: with keras preprocessing **

1- Densenet with imagenet and didnt freeze (trained over them)
71% train and 68% validation

epoch=6 mins

2- Densenet WITHOUT imagenet and didnt freeze 

62% train and 59% validation

epoch=5.5 mins

3-InceptionV3 with imagenet and didnt freeze (trained over them)
79% train and 74% validation

epoch=5 mins

4-InceptionV3 WITHOUT imagenet and didnt freeze 
59% train and 57% validation

epoch=5 mins

**Shoulder data: with keras preprocessing **

1-Densenet with imagenet and no freeze
51% train and 51% Validation

2-Mobilenet with imagenet and no freeze
51% train and 52% Validation

3-Mobilenet with imagenet and no freeze 2 DENSES INSTEAD OF 4

50% train and 50% Validation
 also 51% train and 50 Validation
 
**Finger data: with keras preprocessing **

1-Densenet with imagenet and no freeze
63% train and 69% Validation

**Elbow data: with keras preprocessing **

1-Densenet with imagenet and no freeze
59% train and 50% Validation

**Humerus data: with keras preprocessing **


1-Densenet with imagenet and no freeze
47% train and 49% Validation

**Hand data: with keras preprocessing **

1-Densenet with imagenet and no freeze
73% train and 59% Validation

**Forearm data: with keras preprocessing **

1-Densenet with imagenet and no freeze
64% train and 50% Validation


** Conclusions till now **

1- keras preprocessing (nomalization and subtracting mean maybe ) with inception reduces  accuracy both training and validation , but with DenseNet it improved accuracy 

2- Training over imagenet weights without freezing the TL model gives significantly higher accuracy than freezing whole TL and just training denses

3-Training TL without imagenet and no freeze till now (2 Trials) seems to get worse results than with imagenet and training over it

Mobilenet Preprocess

{'FINGER': [0.6145710928319624, 0.4642082429501085], 

'ELBOW': [0.4877306834585501, 0.48817204329916225], 

'HUMERUS': [0.4716981132075472, 0.4826388888888889], 

'HAND': [0.7322749413674905, 0.5891304347826087], '

FOREARM': [0.36219178082191783, 0.5016611295681063]}


Mobilenet no preprocess

{'FINGER': [0.5084214651299793, 0.5422993490791631],

'ELBOW': [0.4068140336645711, 0.4946236559139785], '

HUMERUS': [0.47091194968553457, 0.4861111111111111]

'HAND': [0.7322749413674905, 0.5891304347826087]

'FOREARM': [0.41315068509480724, 0.4983388704318937]

'SHOULDER': [0.4970760233544666, 0.5062166962699822]}

**Takeaway : preprocessing USUALLY doesnt matter in accuracy**

Mobilenet no preprocess no imagenet no freeze

{'FINGER': [0.428515472011243, 0.46637744034707157],

'ELBOW': [0.4068140336645711, 0.4946236559139785],

'HUMERUS': [0.47091194968553457, 0.4861111111111111],

'HAND': [0.7322749413674905, 0.5891304347826087],

'FOREARM': [0.6284931512074928, 0.5016611295681063],

'SHOULDER': [0.5177228787674057, 0.5062166962699822],

'WRIST': [0.5384615386454278, 0.5523520485584219]}

Mobilenet preprocess, imagenet, froze all

(highest finger,same humerus, same hand, worse same forearm)


{'FINGER': [0.6549157851404458, 0.6811279839393632], 

'ELBOW': [0.41188399917936197, 0.5161290323381783],

'HUMERUS': [0.47091194968553457, 0.4861111111111111], 

'HAND': [0.7322749413674905, 0.5891304347826087],

'FOREARM': [0.36219178082191783, 0.5016611295681063]}